# Endpoint


## Custom Container
- https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb
- https://github.com/aws/amazon-sagemaker-examples/tree/70d51f2f2f5a02c40b7b5f8534adf671c2215e09/advanced_functionality/scikit_bring_your_own/container/decision_trees
- serve파일
    - 맨처음 실행됨. gunicorn을 실행해서, wsgi.py를 실행
    - '-k', 'gevent' 를 하면 ping에서 더 좋은 듯. http://blog.hwahae.co.kr/all/tech/tech-tech/5567/
- predictor.py
    - api호출시 여기의 api 함수가 호출됨


## Custom model
- https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/xgboost_bring_your_own_model/xgboost_bring_your_own_model.ipynb
- 모델파일은 tar.gz로 압축가능. sagemaker 모델 생성시 설정된 압축파일은 자동으로 압축 해제 함



In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

boto3_sess = boto3.Session()
sess = sagemaker.session.Session(boto3_sess)
sm = boto3_sess.client("sagemaker")
sm_runtime = boto3_sess.client("sagemaker-runtime")
region = boto3_sess.region_name
role = get_execution_role()
bucket = sess.default_bucket()

print(boto3.__version__)
print(sagemaker.__version__)
print(region)
print(role)
print(bucket)

In [ ]:
model_name = "some-model"

## Create Endpoint

In [ ]:
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_name = "some-model-endpoint-config-{}".format(timestamp_prefix)
print(endpoint_config_name)

In [ ]:
sess.create_endpoint_config(
    name=endpoint_config_name,
    model_name=model_name,
    initial_instance_count=1,
    # instance_type="ml.m5.large",
    instance_type="ml.t2.medium"
)

In [ ]:
endpoint_name = "some-model-v1-stage"
sess.create_endpoint(endpoint_name, endpoint_config_name)
# sess.update_endpoint(endpoint_name, endpoint_config_name)

### (다른 방법) 모델 생성 후, estimator로 endpoint 간단히 생성하기

In [ ]:
# xgb_script_mode_estimator = XGBoost(
#     entry_point=script_path,
#     framework_version="1.3-1",  # Note: framework_version is mandatory
#     hyperparameters=hyperparams,
#     role=role,
#     instance_count=2,
#     instance_type=instance_type,
#     output_path=output_path,
# )
#
# predictor = xgb_script_mode_estimator.deploy(
#     initial_instance_count=1, instance_type="ml.m5.2xlarge"
# )

## Invoke end point

### 1st way
- predictor.predict 사용
- 단일 json input 처리

import json

with open("sample-request.json", "r") as j:
    request_dict = json.load(j)
request_data = request_dict
payload = json.dumps(request_dict)

In [ ]:
%%time
from sagemaker.predictor import Predictor

# for json
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=sagemaker.serializers.IdentitySerializer(content_type="application/json"),
    deserializer=sagemaker.deserializers.JSONDeserializer()
)

response = predictor.predict(payload)
response

# {
#   "label" : '1'
# }

In [ ]:
### 2nd way
- invoke_endpoint() 사용
- input 여러개 처리


runtime_client = boto3.client("runtime.sagemaker", region_name=region)
response = runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name, ContentType="text/libsvm", Body=payload
)
result = response["Body"].read().decode("ascii")
print("Predicted values are {}.".format(result))
# 여러 input을 한번에 처리 가능

In [ ]:
## Create Endpoint for release

endpoint_name = "some-model-release"
# sess.create_endpoint(endpoint_name, endpoint_config_name)
# sess.update_endpoint(endpoint_name, endpoint_config_name)

In [ ]:
## Delete Endpoint

from sagemaker.predictor import Predictor
predictor = Predictor(
    endpoint_name=endpoint_name
)
predictor.delete_endpoint()

predictor.delete_endpoint()

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor.delete_endpoint()